In [ ]:
import json
import os

# Load environment variables
with open("local.settings.json") as f:
    os.environ.update(json.load(f)["Values"])

In [ ]:
import json
import os
from azure.storage.blob import BlobServiceClient
from shapely import wkt
from shapely.geometry import mapping
import cuid
import pandas as pd


def wkt_to_geojson(wkt_str):
    try:
        return mapping(wkt.loads(wkt_str))
    except Exception:  # For production, catch a more specific exception
        return None

blob_service_client = BlobServiceClient.from_connection_string(os.environ["SAFEGRAPH_CONNECTION_STRIMG"])
container_client = blob_service_client.get_container_client(os.environ["SAFEGRAPH_CONTAINER_NAME"])
blobs = [
    blob for blob in container_client.list_blobs(name_starts_with=os.environ["SAFEGRAPH_BLOB_PREFIX"])
    if blob.name.endswith(".gz")
]

df = (
    pd.concat(
        [
            pd.read_csv(
                container_client.get_blob_client(blob).download_blob(),
                compression="gzip",
                usecols=[0, 3, 10, 11, 12, 13, 17, 25],
            )
            .dropna(
                subset=[
                    "location_name",
                    "placekey",
                    "street_address",
                    "city",
                    "region",
                    "postal_code",
                    "polygon_wkt",
                ]
            )
            .rename(
                columns={
                    "location_name": "title",
                    "placekey": "ESQID",
                    "street_address": "street",
                    "region": "state",
                    "postal_code": "zipCode",
                    "polygon_wkt": "polygon",
                    "related_parking": "relatedGeoFrames",
                }
            )
            .assign(
                id=lambda x: [cuid.cuid() for _ in range(len(x))],
                polygon=lambda x: x["polygon"].apply(
                    lambda y: json.dumps(
                        {
                            "type": "FeatureCollection",
                            "features": [
                                {"type": "Feature", "geometry": wkt_to_geojson(y)}
                            ],
                        }
                    )
                ),
                related_parking=lambda x: x["related_parking"].apply(json.loads),
                source="SafeGraph",
                zipCode=lambda x: x["zipCode"].apply(lambda y: str(int(y)).zfill(5)),
            )
            for blob in blobs
            if f.endswith(".gz")
        ],
        ignore_index=True,
    )[
        [
            "id",
            "title",
            "source",
            "ESQID",
            "street",
            "city",
            "state",
            "zipCode",
            "polygon",
            "relatedGeoFrames"
        ]
    ]
)

In [ ]:
df.to_csv("all_data.csv", index=False)

In [ ]:
import psycopg2
import urllib.parse

dsn = os.environ["DATABIND_SQL_KEYSTONE"]

# Remove the SQLAlchemy prefix
if dsn.startswith("postgresql+psycopg2://"):
    dsn = dsn.replace("postgresql+psycopg2://", "")

# Prepend a scheme for proper URL parsing
parsed = urllib.parse.urlparse("postgresql://" + dsn)

csv_file = "all_data.csv"

with psycopg2.connect(
    dbname=parsed.path[1:],  # Remove leading slash
    user=parsed.username,
    password=parsed.password,
    host=parsed.hostname,
    port=parsed.port,
) as conn:
    with conn.cursor() as cur:
        # 1. Create a temporary staging table.
        # Adjust the column types if necessary. Here we assume polygon is stored as JSONB.
        cur.execute(
            """
            CREATE TEMP TABLE staging_table (
                id TEXT,
                source TEXT,
                "ESQID" TEXT,
                title TEXT,
                street TEXT,
                city TEXT,
                state TEXT,
                "zipCode" TEXT,
                polygon JSONB
            )
        """
        )
        conn.commit()
        print("Table created.")

        # 2. Load CSV data into the staging table using the COPY command.
        with open(csv_file, "r") as f:
            # Skip the header row if your CSV includes it.
            next(f)
            cur.copy_expert(
                sql="""
                COPY staging_table (id, title, source, "ESQID", street, city, state, "zipCode", polygon)
                FROM STDIN WITH CSV QUOTE '\"'
                """,
                file=f,
            )
        conn.commit()
        print("Data copied")

        # 3. Upsert data into the target table.
        # This statement inserts new records and updates existing rows based on the primary key (id).
        cur.execute(
            """
            INSERT INTO keystone."TargetingGeoFrame" (id, title, source, "ESQID", street, city, state, "zipCode", polygon)
            SELECT DISTINCT ON ("ESQID") id, title, source, "ESQID", street, city, state, "zipCode", polygon
            FROM staging_table
            ORDER BY "ESQID", id  -- Adjust the order clause to pick the preferred record per ESQID
            ON CONFLICT ("ESQID")
            DO UPDATE SET
                title = EXCLUDED.title,
                street = EXCLUDED.street,
                city = EXCLUDED.city,
                state = EXCLUDED.state,
                "zipCode" = EXCLUDED."zipCode",
                polygon = EXCLUDED.polygon;
        """
        )
        conn.commit()

print("Table updated successfully using the CSV file.")